pip install bs4

pip install requests

In [1]:
import ssl

hostname = 'www.python.org'
context = ssl._create_unverified_context()

pip install lxml

import nltk
nltk.download('punkt')

In [2]:
all_product_values = {}
query = "014100085478"
message = ""

In [3]:
import datetime
import time
now = datetime.datetime.now()
from firebase import firebase
import datetime
#from selenium import webdriver
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import requests
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import json

fb_connect = firebase.FirebaseApplication("https://ecoscanner-cb66d.firebaseio.com/",None)

sw = stopwords.words("english")

later = datetime.datetime.now()
duration = later- now
time_dict = {"imports":duration}


# make sure "v2_organic_operations_dict.json" in directory

In [4]:
def endpoints(x,lower=0,upper=10):
    if x<lower:
        return lower
    elif x>upper:
        return upper
    else:
        return x

In [5]:

## NLP filtration method for products
def check_nlp(itemvarieties,X_list,product,sw=sw):

    # if cosine function doesn't work, use own alternative
    def own_similarity_function(product,itemvarieties):


        if product in itemvarieties:
            return endpoints((len(product)*2/len(itemvarieties)))
        elif itemvarieties in product:
            return endpoints((len(itemvarieties)*2/len(product)))     
        else:
            return 0


    l1 =[];l2 =[] 

    # tokenization 

    Y_list = word_tokenize(itemvarieties) 

    # sw contains the list of stopwords 


    # remove stop words from string 
    X_set = {w for w in X_list if not w in sw} 
    Y_set = {w for w in Y_list if not w in sw} 

    # form a set containing keywords of both strings 
    rvector = X_set.union(Y_set) 
    for w in rvector: 
        if w in X_set:
            l1.append(1) # create a vector 
        else:
            l1.append(0) 
        if w in Y_set:
            l2.append(1) 
        else:
            l2.append(0) 

    c = 0

    # cosine formula 
    for i in range(len(rvector)): 
        c+= l1[i]*l2[i] 
    
    try:
        cosine = c / float((sum(l1)*sum(l2))**0.5)
    except ZeroDivisionError:
        return own_similarity_function(product,itemvarieties)
    
    if cosine ==0:
        return own_similarity_function(product,itemvarieties)
    
    return cosine


In [6]:
## creates a dictionary with all known info from upcitemdb.com

def lookup(query):
    ## append search UPC to the database website
    my_url = "https://www.upcitemdb.com/upc/"+query
    
    #open urllib client, accounting for page not-existing
    try:
        uClient = uReq(my_url, context=context)
    except:
        return None
    
    # grab page html, save as soup
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html,"html.parser")
    
    if "invalid" in page_soup.text:
        return None
    
    
    # find the table of item details, use first bc only 1
    container = page_soup.findAll("table",{"class":"detail-list"})[0]
    
    ## create blank dictionary
    info_dict={}
    
    ## find product name
    
    # climb into blah is associated with blah
    #pieces = str(page_soup.head.findAll("meta")[1].meta).split(",")
    #get rid of first thing
    #pieces.pop(0)
    
    piece = page_soup.findAll("p",{"class":"detailtitle"})[0]
    
    product_name = piece.b.text

    
    def clean(x):
        x=x.replace("&amp","&")
        for char in x:
            if char.isnumeric():
                tokens = x.split(char)
                y = tokens[0]
                break
        if not y:    
            return x.split(",")[0]
        else:
            return y.split(",")[0].strip()

    product_name = clean(product_name)

    
    ## find all other info
    trs = container.findAll("tr")
    
    ## loop through info things
    for tr in trs:
        
        # make list, containing key line and value line
        tds = tr.findAll("td")
        
        ## parse out key and value, save to info dict
        key = str(tds[0]).strip("<td>").strip("</").strip(":")
        value = str(tds[1]).strip("<td>").strip("</").strip().strip("\t")
        # don't care ab last-scanned
        if key != "Last Scanned":
            if key=="Brand":
                for part in value.split():
                    for part2 in part.split(','):
                        product_name = product_name.replace(part2,"")
                
                info_dict["product name"] = product_name.strip()
            info_dict[key]=value
    return info_dict

In [7]:
#print(lookup(query))
#capath="Macintosh HD⁩/Users⁩/⁨mihirheda⁩/⁨Downloads/securly_ca_2034.crt⁩"

In [8]:
#my_url = "https://www.upcitemdb.com/upc/"+query


print(lookup("013562300587"))

#open urllib client, accounting for page not-existing

uClient = uReq("https://www.upcitemdb.com/upc/071641053649")


# grab page html, save as soup
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html,"html.parser")

pieces = str(page_soup.head.findAll("meta")[1].meta).split(",")
pieces.pop(0)
print("pieces before splicing:")
print(pieces)

output_str = ""
a=True
for piece in pieces:    
    if piece==" upc lookup database":
        a=False
    print("piece",piece,"a",a)
    if a:
        output_str+=piece
print(output_str)

In [9]:
now = datetime.datetime.now()
upc_info_dict = lookup(query)
later = datetime.datetime.now()
duration = later-now
time_dict["lookup upc"] = duration

if upc_info_dict is None:
    message += "lookup failed and returned None, barcode not found"
brand = upc_info_dict["Brand"]
all_product_values["brand"] = brand
print(upc_info_dict)


{'UPC-A': '0 14100 08547 8', 'EAN-13': '0 014100 085478', 'Amazon ASIN': 'B004CRNR0C', 'Country of Registration': 'United States', 'product name': 'Goldfish', 'Brand': 'Pepperidge Farm', 'Model #': 'Cheddar Goldfish', 'Size': '6.6-ounce bag', 'Color': 'Stainless steel', 'Weight': '0.5 Pounds', 'Product Dimension': '4 X 2.8 X 5.8 inches'}


In [10]:
def clean_product(product,brand):
    sucky_chars = ["-",":",";","(",")","_","%","#","^","&","!","oz","Oz"]
    for char in sucky_chars:
        product = product.replace(char,"").strip()
    for brand_part in brand.split(","):
        product = product.replace(brand_part,"").strip()
    return product

print(clean_product("Annie's Low Sodium Mac Mild Cheddar Cheese 6 oz","Annie's"))

In [11]:
print(upc_info_dict)

{'UPC-A': '0 14100 08547 8', 'EAN-13': '0 014100 085478', 'Amazon ASIN': 'B004CRNR0C', 'Country of Registration': 'United States', 'product name': 'Goldfish', 'Brand': 'Pepperidge Farm', 'Model #': 'Cheddar Goldfish', 'Size': '6.6-ounce bag', 'Color': 'Stainless steel', 'Weight': '0.5 Pounds', 'Product Dimension': '4 X 2.8 X 5.8 inches'}


In [12]:
product_name = clean_product(upc_info_dict["product name"],upc_info_dict["Brand"])

In [13]:
print(product_name)

Goldfish


In [14]:
all_product_values["product name"] = product_name
product = product_name

In [15]:
def find_company(brand):
    
    result = fb_connect.get(f"/brand-comp/{brand}",None)
    if result!=None:
        return result

    def side_box_check(brand,page_soup):
        parents = []
        side_box_list = page_soup.findAll("div",{"class":"zloOqf PZPZlf"})
        for item in side_box_list:
            if "Parent" in str(item):
                mini_set = item.findAll("span",{"class":"LrzXr kno-fv"})
                mini_set = str(mini_set[0]).split(",")
                for f1 in mini_set:
                    for snippet in str(f1).split(">"):
                        if "</a" in snippet:
                            parents.append(snippet.strip("</a").replace("&amp;","&"))
        if len(parents)!=0:
            return parents
        else:
            return table_check(brand,page_soup)
        
    def table_check(brand,page_soup):
        try:
            table = page_soup.findAll("div",{"class":"webanswers-webanswers_table__webanswers-table"})[0]
        except:
            return lame_box_check(brand,page_soup)
        rows = table.findAll("tr")
        for tr in rows:
            if "Owner" in str(tr):
                owner_line = str(tr.findAll("td")[1])
                trim1 = owner_line.split(">")[1]
                trim2 = trim1.strip("</td>")
                return trim2.split(",")
    
    def lame_box_check(brand,page_soup):
        output = []
        not_comp = ["company",brand,"parent"]
        try:
            info = page_soup.findAll("span",{"class":"e24Kjd"})[0]
        except IndexError:
            return brand
        bolded = info.findAll("b")
        for bold in bolded:
            bold = str(bold).strip("<b>").strip("</b>")
            if bold not in not_comp:
                output.append(bold)
        for option in output:
            if option in brand or brand in option:
                output.pop(output.index(option))
        return output

    
    my_url = f"https://www.google.com/search?q={brand}+parent+company"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"

    headers = {"user-agent":user_agent}
    resp = requests.get(my_url, headers=headers)

    if resp.status_code == 200:
        page_soup = soup(resp.content, "html.parser")
    else:
        return brand
    
    list_tag = page_soup.findAll("a",{"class":"FLP8od"})
    if len(list_tag)==0:
        list_tag = page_soup.findAll("div",{"class":"Z0LcW AZCkJd"})
        if len(list_tag)==0:
            list_tag = page_soup.findAll("div",{"class":"Z0LcW"})
            if len(list_tag)==0:
                output = []
                intake = side_box_check(brand,page_soup)
                if isinstance(intake,str):
                    return intake
                for item in intake:
                    output.append(item.replace("&amp;","&"))
                return output
    full_tag = str(list_tag[0])
    splitted = full_tag.split(">")
    pre_company = splitted[1]
    post_company = pre_company.split("</")[0]
    answer = post_company.replace("&amp;","&")
    result = fb_connect.put("/brand-comp/",brand,answer)
    return answer



In [16]:
now = datetime.datetime.now()
owner_company = find_company(brand)
later = datetime.datetime.now()
duration = later-now
time_dict["owner company"] = duration
all_product_values["owner company"] = owner_company
if type(owner_company)=="list":
    owner_company=owner_company[0]

In [17]:
def multi_splits(lst1,splitters_list):
    def split_single(lst2,splitter):
        out = []
        for thing in lst2:
            out.extend([x.strip() for x in thing.split(splitter)])
        return out
    
    for splitter in splitters_list:

        lst1 = split_single(lst1,splitter)
    
    return lst1
        
    

In [18]:

#splitters = [",",":",".","and/or","and","or"," of "]


# Ingredients

def find_ingredients(product,brand,company):

    all_results=[]
    my_api_key = "9ZTbuCeX9dP5keqlGJZyWNdg1c1xCu3p9pLhJ1MS"
    query = product.replace(" ","%20") # format more friendly
    # make request with product name, brand owner, limit to 25 results
    response = requests.get(f"https://api.nal.usda.gov/fdc/v1/foods/search?api_key={my_api_key}&query={query}&brandOwner={brand}&pageSize=25")
    big_dict = response.json() # json format
    foods = big_dict["foods"] # list of all foods that match result
    
    # if less than 10 foods match search with that brand, check again replacing brand with company
    if len(foods)<10:
        response2 = requests.get(f"https://api.nal.usda.gov/fdc/v1/foods/search?api_key={my_api_key}&query={query}&brandOwner={company}&pageSize=25")
        big_dict_2 = response.json()
        foods2 = big_dict["foods"]
        foods.extend(foods2)
        
    
    ingredients = set([]) #initialize empty set
    for food in foods:
        if "ingredients" in food.keys(): # if food has ingredients, get the string
            string_mini_ingred = food["ingredients"]
            ingreds = multi_splits(string_mini_ingred.split("("),splitters)
            for half in ingreds:
                if "FLAVOR" not in half: # get rid of "Natural and Artifical flavor", "maintains flavor", etc
                    ingredients.add(half.strip().strip(")")) #clean up and add to ingredients set
    
    # cleaning blank strings out of list
    x =[]
    for ingredient in ingredients:
        ingredient = ingredient.strip()
        if ingredient!="" and ingredient!=" " and len(ingredient)>3:
            a = True
            if " " in ingredient:
                if len(ingredient[0:ingredient.index(" ")])<0 or len(ingredient[ingredient.index(" "):-1])<2:
                    a = False
            if a:
                trash = ["[","]","*","/","organic","for","with"]
                nutrient = ingredient
                nutrient = nutrient.lower()
                for stink in trash:
                    nutrient = nutrient.replace(stink,"")
                nutrient = nutrient.strip()
                x.append(nutrient)
    

    return x
            
        #keys = food.keys()
        #if "ingredients" in keys:
            #ingred_str = food["ingredients"]
            #all_matches.append({"name":food["description"],"ingredients":ingred_str})
    #return all_matches

now = datetime.datetime.now()

nutrients = find_ingredients(product,brand,owner_company)

later = datetime.datetime.now()
duration = later- now
time_dict["find ingredients"] = duration

#now = datetime.datetime.now()

## initialize selenium
#chromedriver = "C:\\Users\\14082\\Documents\\Random Projects\\LancerHacks\\chromedriver83"
#driver = webdriver.Chrome(chromedriver)

#later = datetime.datetime.now()
#duration = later- now
#time_dict["selenium initialization"] = duration

driver = None

def score_nutrient(nutrient,driver,fb_connect=fb_connect):

    print(nutrient)
    
    # check if score already in firebase

    try:
        result = fb_connect.get(f"/ingredient scores/{nutrient}",None)
    except:
        result = None
        print("nutrient was",nutrient)
        
    if result is not None:
        if "N/A" in result:
            return ((0,"none"),driver)
        print(result,type(result))
        result_parts = result.split(",") # split the score and source
        result_parts[0] = float(result_parts[0]) #turn the first part into a float
        result_parts[1] = result_parts[1].strip() # get rid of space in source string
        
        return (tuple(result_parts),driver)

    if driver is None: 
        # replace with location of chromedriver app on VM
        # https://chromedriver.chromium.org/downloads
        ## initialize selenium
        chromedriver = "C:\\Users\\14082\\Documents\\Random Projects\\LancerHacks\\chromedriver83"
        driver = webdriver.Chrome(chromedriver)

        later = datetime.datetime.now()
        duration = later- now
        time_dict["selenium initialization"] = duration

    
    try:
        driver.get(f"https://comptox.epa.gov/dashboard/dsstoxdb/results?search={nutrient}#toxicity-values")
        
        # find the label containing eco radio button, which will be visible; if error finding, page doesn't exist
        label = driver.find_element_by_css_selector("label[class='b-radio radio button is-medium']")
    except:
        #driver.close()
        result = fb_connect.put(f"/ingredient scores",nutrient,"N/A") # add in firebase, don't waste time checking again
        return ((0,"none"),driver)

    # empty string for hazard values
    hazard_vals = []
    
    if not label.get_attribute("disabled"): # check that eco is not disabled (arsenic example)
        try:
            label.click()
        except:
            exit = driver.find_element_by_css_selector("button[title='No thanks']")
            exit.click()
            label.click()
        source = "eco"
    else:
        source="human"
    
    body = driver.find_element_by_css_selector("tbody")
    rows = body.find_elements_by_css_selector("tr")
    for row in rows:
        cells = row.find_elements_by_css_selector("td")
        hazard_vals.append(int(cells[1].text)) # convert text from second cell per row into int, append
    
    hazard_vals.sort(reverse=True) #sort in descending order
    hazard_vals = hazard_vals[0:3] #take only 3 highest scores
    ave = sum(hazard_vals)/len(hazard_vals) #find ave of scores
    score = ave-4 #subtract 4
    
    #driver.close() # close the browser
    out_tup = (score,source) # conversion to tuple
    
    # save to firebase
    result = fb_connect.put(f"/ingredient scores",nutrient,str(out_tup).strip(")").strip("(").replace("\'",""))

    return (out_tup,driver)

total_score = 0
sources = []

now = datetime.datetime.now()

# for each ingredient in the list
for nutrient in nutrients:
    print(type(driver))
    nutrient_score,driver = score_nutrient(nutrient,driver) # find the score 
    print(type(driver))
    total_score+=nutrient_score[0] # add to total score
    sources.append(nutrient_score[1]) # add source to a list of sources
    
later = datetime.datetime.now()
duration = later- now
time_dict["score all ingredients"] = duration
    
num_used = len(sources)-sources.count("none") # find the num of scores that aren't none
ave_score = total_score/num_used 

# set upper and lower cap
ave_score = endpoints(ave_score,lower=0,upper=4)

ingredient_end_score = ave_score * -1
all_product_values["ingredient score"] = ingredient_end_score
print(ingredient_end_score)

if driver is not None:
    now = datetime.datetime.now()
    driver.close()
    later = datetime.datetime.now()
    duration = later- now
    time_dict["close selenium"] = duration

## TO DOWNLOAD ZIP FILE

response = requests.get(f"https://organicapi.ams.usda.gov/IntegrityPubDataServices/OidPublicDataService.svc/rest/GetAllOperationsPublicData?api_key={my_api_key}",stream=True)
target_path = 'organic_ops.zip'
handle = open(target_path, "wb")
for chunk in response.iter_content(chunk_size=512):
    if chunk:  # filter out keep-alive new chunks
        handle.write(chunk)
handle.close()

import requests
from bs4 import BeautifulSoup as soup

my_api_key = "9ZTbuCeX9dP5keqlGJZyWNdg1c1xCu3p9pLhJ1MS"
json = {"startldx":"1","count":"1000","countries":["USA"]}
response = requests.post(f"https://organicapi.ams.usda.gov/IntegrityPubDataServices/OidPublicDataService.svc/rest/Operations?api_key={my_api_key}",json=json)
all_text = response.text

page_soup = soup(all_text,"lxml")
page2 = soup(str(page_soup),"html.parser")
operations_list = page2.html.body.operationsresult.operations.findAll("a:operation")

state_dict = {}
def midinfo(x):
    x = str(x)
    return x.split(">")[1].split("<")[0]
for operation in operations_list:
    operationname = midinfo(operation.findAll("a:operationname")[0])
    operationstatus = midinfo(operation.findAll("a:nopoperationstatus")[0])
    addresses = operation.findAll("a:addresses")
    address = addresses[0].findAll("a:address")
    state_untrimmed = str(address[0].findAll("a:stateorprovince")[0])
    state = midinfo(state_untrimmed)
    print(operationname,operationstatus,state)
    if state in state_dict.keys():
        state_dict[state]+=1
    else:
        state_dict[state]=1
print(state_dict)

all_text = """    <Operation>
      <op_certifierName>[OTCO] Oregon Tilth Certified Organic</op_certifierName>
      <op_nopOpID>8150000053</op_nopOpID>
      <op_name>Wong Potatoes, Inc</op_name>
      <op_clientID>OT-017900</op_clientID>
      <op_contFirstName>Daniel</op_contFirstName>
      <op_contLastName>Chin</op_contLastName>
      <op_status>Certified</op_status>
      <op_statusEffectiveDate>2014-11-06</op_statusEffectiveDate>
      <op_nopAnniversaryDate>2020-04-01</op_nopAnniversaryDate>
      <op_lastUpdatedDate>2019-05-15</op_lastUpdatedDate>
      <opSC_HANDLING>Certified</opSC_HANDLING>
      <opSC_HANDLING_ED>2014-11-06</opSC_HANDLING_ED>
      <op_phone>(541) 798-5353</op_phone>
      <op_email>chinfarms@gmail.com</op_email>
      <opMA_line1>17600 Hwy 39</opMA_line1>
      <opMA_city>Klamath Falls</opMA_city>
      <opMA_state>Oregon</opMA_state>
      <opMA_country>United States of America (the)</opMA_country>
      <opMA_zip>97603</opMA_zip>
    </Operation>
    <Operation>
      <op_certifierName>[OTCO] Oregon Tilth Certified Organic</op_certifierName>
      <op_nopOpID>8150000052</op_nopOpID>
      <op_name>High Mountain LLC</op_name>
      <op_clientID>OT-004440</op_clientID>
      <op_contFirstName>Kevin</op_contFirstName>
      <op_contLastName>Christensen</op_contLastName>
      <op_status>Suspended</op_status>
      <op_statusEffectiveDate>2011-11-16</op_statusEffectiveDate>
      <op_lastUpdatedDate>2017-12-27</op_lastUpdatedDate>
      <opSC_CR>Suspended</opSC_CR>
      <opMA_line1>PO Box 968</opMA_line1>
      <opMA_city>Mattawa</opMA_city>
      <opMA_state>Washington</opMA_state>
      <opMA_country>United States of America (the)</opMA_country>
      <opMA_zip>99349</opMA_zip>
    </Operation>
    <Operation>
      <op_certifierName>[OTCO] Oregon Tilth Certified Organic</op_certifierName>
      <op_nopOpID>8150000051</op_nopOpID>
      <op_name>Marvin Lynch</op_name>
      <op_otherNames>DBA O'lynch Dairy</op_otherNames>
      <op_clientID>OT-007458</op_clientID>
      <op_contFirstName>Marvin</op_contFirstName>
      <op_contLastName>Lynch</op_contLastName>
      <op_status>Certified</op_status>
      <op_statusEffectiveDate>2009-05-04</op_statusEffectiveDate>
      <op_nopAnniversaryDate>2020-01-01</op_nopAnniversaryDate>
      <op_lastUpdatedDate>2019-05-15</op_lastUpdatedDate>
      <opSC_CR>Certified</opSC_CR>
      <opSC_CR_ED>2009-05-04</opSC_CR_ED>
      <opSC_LS>Certified</opSC_LS>
      <opSC_LS_ED>2009-05-04</opSC_LS_ED>
      <op_phone>(563) 852-5285</op_phone>
      <op_email>marvlynch@yahoo.com</op_email>
      <opMA_line1>24764 Hwy 151 W</opMA_line1>
      <opMA_city>Cascade</opMA_city>
      <opMA_state>Iowa</opMA_state>
      <opMA_country>United States of America (the)</opMA_country>
      <opMA_zip>52033</opMA_zip>
    </Operation>

"""


huge_dict={}
all_text = all_text.replace("\n","")
all_operations = all_text.split("</Operation>")
concerns = ["op_nopOpID","op_name","opMA_state"]
for operation in all_operations:
    if "United States" in operation and "status>Certified<" in operation:
        internal_dict = {}
        deets = operation.split("</")
        for deet in deets:
            for concern in concerns:
                if "<"+concern+">" in deet:
                    internal_dict[concern]=deet.split(">")[-1]
        huge_dict[internal_dict["op_name"]] = internal_dict

##CREATING DICTIONARY OF ALL ORGANIC OPERATIONS

all_text = ""
op_file = open("organic_ops_file.txt","r",encoding="utf-8")
for line in op_file.readlines():
    all_text+=line.strip("\n")
op_file.close()

huge_dict={}
all_text = all_text.replace("\n","")
all_operations = all_text.split("</Operation>")
concerns = ["op_nopOpID","op_name","opMA_state"]
for operation in all_operations:
    if "United States" in operation and "status>Certified<" in operation:
        internal_dict = {}
        deets = operation.split("</")
        for deet in deets:
            for concern in concerns:
                if "<"+concern+">" in deet:
                    internal_dict[concern]=deet.split(">")[-1]
        huge_dict[internal_dict["op_name"]] = internal_dict


## WRITING INTO JSON FILE

import json

json_format = json.dumps(mega_dict)
f = open("v2_organic_operations_dict.json","w")
f.write(json_format)
f.close()

In [19]:
#Creating MegaDict

def create_dict(filename="v2_organic_operations_dict.json"):    
    with open(filename,"r") as json_file:
        mega_dict = json.load(json_file)
        if type(mega_dict)=="str":
            print("oops string error")
        else:
            return mega_dict


## trying to fix capitalization in the dictionary
import string
mega_dict = {}
for key in huge_dict.keys():
    if key.isupper():
        fixed = string.capwords(key)
        huge_dict[key]["op_name"] = fixed
        mega_dict[fixed] = huge_dict[key]
    else:
        mega_dict[key] = huge_dict[key]
print(mega_dict)

In [20]:
#try:
 #   y = mega_dict
#except NameError:
 #   mega_dict = create_dict()


now = datetime.datetime.now()    
opID_dict = create_dict()
later = datetime.datetime.now()
duration = later- now
time_dict["creating opID_dict"] = duration
    
def find_operation_id(brand,owner_company=owner_company,opID_dict=opID_dict):
    
    # first check firebase
    result = fb_connect.get(f"/brand-opID/{brand}",None)
    if result is not None:
        return result
    
    
    brand = brand.replace(",","")
    found_keys = list()
    dict_keys=opID_dict.keys()
    for key in dict_keys:
        if brand in key or key in brand:
            found_keys.insert(0,[key,opID_dict[key]["op_nopOpID"]]) #make first term in found_keys a list with key, and opID
    if len(found_keys)>=1:
        
        # default input to similarity function, only need to tokenize and find stopwords once
        X_list = word_tokenize(brand) 

        for hit in found_keys:
            sim_dict = {}
            name = hit[0]
            similarity_sc
            similarity_score = check_nlp(name,X_list,brand)
            sim_dict[similarity_score] = hit
        max_sim = max(sim_dict.keys())
        if max_sim>=0.3:
            found_keys = sim_dict[max_sim]
        else:
            found_keys = []

    else:
        if owner_company!="":
            found_keys = find_operation_id(owner_company,owner_company="")
    if len(found_keys)==0:
        opID = "None"
    elif isinstance(found_keys,list) and len(found_keys)>0:
        if isinstance(found_keys[0],str):
            opID = found_keys[1]
        else:
            opID = found_keys[0][1]
    else:
        opID = found_keys
    response = fb_connect.put("/brand-opID",brand,opID)
    return opID

In [21]:
#opID = find_operation_id(brand,owner_company)

now = datetime.datetime.now()
opID = find_operation_id(brand,owner_company)
later = datetime.datetime.now()
duration = later- now
time_dict["find opID"] = duration

all_product_values["organic operation ID"] = opID


In [22]:
#print(opID)

## Receiving Operation Products based on op_code

import requests
from bs4 import BeautifulSoup as soup
#
op_id = "5561001956"
#
my_api_key = "9ZTbuCeX9dP5keqlGJZyWNdg1c1xCu3p9pLhJ1MS"
json = {"startIdx":"1","count":"1000","operationId":op_id}
response = requests.post(f"https://organicapi.ams.usda.gov/IntegrityPubDataServices/OidPublicDataService.svc/rest/Items?api_key={my_api_key}",json=json)
# parsing into a list of items
all_text = response.text
page_soup = soup(all_text,"lxml")
page2 = soup(str(page_soup),"html.parser")

items = page2.findAll("a:item")

if len(items)==0:
    return "Error"

In [23]:

## Receiving Operation Products based on op_code

def find_itemvarieties(item,tag="itemvarieties"):
    deets = item.split("</")
    for deet in deets:
        if f"<a:{tag}>" in deet:
            itemvarieties = deet.split(">")[-1]
            return itemvarieties
        
def organic_info(op_id,product,brand):
        
    
    do_trash=True
    if do_trash:
        # get rid of common words that are clutter, this list should grow with testing
        trash_list = ["Organic ",brand,"Super ","Green ","Ounce","ounce","count","Count"]
        for stink in trash_list:
            product = product.replace(stink,"").strip()
        if " " in product:
            for word in product.split():
                for stink in trash_list:
                    if stink in word or word.isnumeric():
                        product = product.replace(word,"").strip()
        product = product.split(",")[0]
        product = product.replace(brand,"").strip()
    
    # default input to similarity function, only need to tokenize and find stopwords once
    X_list = word_tokenize(product) 

    ## API request
    my_api_key = "9ZTbuCeX9dP5keqlGJZyWNdg1c1xCu3p9pLhJ1MS"
    json = {"startIdx":"1","count":"100","operationId":op_id}
    response = requests.post(f"https://organicapi.ams.usda.gov/IntegrityPubDataServices/OidPublicDataService.svc/rest/Items?api_key={my_api_key}",json=json)

    # parsing into a list of items
    all_text = response.text
    page_soup = soup(all_text,"lxml")
    page2 = soup(str(page_soup),"html.parser")
    items = page2.findAll("a:item")

    if len(items)==0:
        return None

    # TEST INFORMATION
    #company = "Forager Project"
    #brand = "Forager Project"
    #product = "Super Green Leafy Greens Chips"
    #

    # create list of API response terms we care about
    concerns = ["itemname","itemvarieties","madewithorganic","organic","organic100","otheritems"]
    
    possible_matches = []
    best_choice = {"similarity score":0.2} # initial threshold for any match
    for item in items:
        cont = True
        item = str(item)
        if "status>Certified<" in item:
            #try:
            itemvarieties = find_itemvarieties(item)
            if itemvarieties=="":
                itemvarieties = find_itemvarieties(item,tag="otheritems")
            if itemvarieties =="":
                itemvarieties = find_itemvarieties(item,tag="itemname")

            similarity_score = check_nlp(itemvarieties,X_list,product)
                
            if similarity_score>best_choice["similarity score"]:
                best_choice = {"similarity score":similarity_score}
                deets = item.split("</")
                for deet in deets:
                    for concern in concerns:
                        inclusion_str = "<a:"+concern
                        exclusion_str = "OASNFOIAJDFOAJFDOINOANEFADUB"
                        if concern=="organic":
                            exclusion_str = "organic100"
                        if inclusion_str in deet and exclusion_str not in deet:
                            best_choice[concern]=deet.split(concern)[-1].strip(">").replace("&amp;","&")
    return best_choice

In [24]:
#print(organic_info("5520336912","Lemon",""))

In [25]:
if opID=="None":
    organic = 0
else:
    
    organic = 1 # default for if the rest doesn't work
    
    ## come here and write the actual function to produce organic score
    now = datetime.datetime.now()
    organic_info_var = organic_info(opID,product,brand)
    later = datetime.datetime.now()
    duration = later- now
    time_dict["find organic info"] = duration
    all_product_values["organic info"] = organic_info_var
    try:
        if "true" == organic_info_var["organic100"]:
            organic = 3
    except:
        print("hit except 1")
        y=0
    try:
        if "true" == organic_info_var["organic"]:
            organic = 2
    except:
        print("hit except 2")
        y=0
    try:
        if "true" == organic_info_var["madewithorganic"]:
            print("worked where it was supposed to")
            organic = 1
    except:
        print("hit except 3")
        y=0

all_product_values["organic"] = organic

    

#####    # get rid of common words that are clutter, this list should grow with testing
    trash_list = ["Organic ",brand,company,"Super ","Green ","Ounce","ounce","count","Count"]
    for stink in trash_list:
        product = product.replace(stink,"").strip()
    for word in product.split():
        for stink in trash_list:
            if stink in word or word.isnumeric():
                product = product.replace(word,"").strip()

    #print("product before recombo is",product)

    # creating different product variations to test against
    product_combos = [product]    
    for word in product.split():
        product_combos.append(product.replace(word,"").strip())

In [26]:
#organic_info("8150000355","Annie's Homegrown Organic Pretzel Bunnies, 7 oz, Pack of 12","Annie's Inc.")

pip install nltk

import nltk


nltk.download("all")

# Program to measure similarity between 
# two sentences using cosine similarity. 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

product = "Super Green Leafy Greens Chips"

# default input to function, only need to tokenize and find stopwords once
X_list = word_tokenize(product) 
sw = stopwords.words('english') 

def compare_terms(itemvarieties,X_list=X_list,sw=sw):
    
    l1 =[];l2 =[] 

    # tokenization 

    Y_list = word_tokenize(itemvarieties) 

    # sw contains the list of stopwords 


    # remove stop words from string 
    X_set = {w for w in X_list if not w in sw} 
    Y_set = {w for w in Y_list if not w in sw} 

    # form a set containing keywords of both strings 
    rvector = X_set.union(Y_set) 
    for w in rvector: 
        if w in X_set:
            l1.append(1) # create a vector 
        else:
            l1.append(0) 
        if w in Y_set:
            l2.append(1) 
        else:
            l2.append(0) 

    c = 0

    # cosine formula 
    for i in range(len(rvector)): 
        c+= l1[i]*l2[i] 
    cosine = c / float((sum(l1)*sum(l2))**0.5) 
    return cosine


import requests

product = "Superco Green Cleaner & Degreaser"
safer_choice_url = f"https://enviro.epa.gov/enviro/efservice/T_SAFERCHOICE/PRODUCT NAME/CONTAINING/{product}/JSON"
response = requests.get(safer_choice_url)

dictionary = response.json()

print(dictionary)

# GREEN SEAL by Product 

"""
def find_greenseal(product):
   
    # default input to similarity function, only need to tokenize and find stopwords once
    X_list = word_tokenize(product) 

    # create search url, define user agent, make request
    search_product = product.replace(" ","%20")
    my_url = f"https://www.greenseal.org/api/search/{search_product}?service_type="
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
    headers = {"user-agent":user_agent}
    resp = requests.get(my_url, headers=headers)

    # if succesful, employ bs4; return None of get request not success
    if resp.status_code == 200:
        page_soup = soup(resp.content, "html.parser")
    else:
        return None
    
    # convert big json response to dictionary and enter results list
    results_dict = json.loads(page_soup.text)
    results_list = results_dict["results"]    
    
    # return None if results is empty
    if len(results_list)==0:
        return None
    
    # initialize empty output list
    item_info = []
    
    # iterate through each result, pulling out info we care about
    for result in results_list:
        mini_dict = {} #empty dictionary to append to list for this product
        product_name = result["name"]
        if len(item_info)>=1: # if  already found hit, start nlp cross checking
            similarity_score = check_nlp(product_name,X_list,product)
            if similarity_score >= 0.3+ 0.05*len(item_info):
                mini_dict["name"] = product_name
                mini_dict["company"] = result["mfg"]
                mini_dict["category"] = result["category"]
                item_info.append(mini_dict)
        else:
                mini_dict["name"] = product_name
                mini_dict["company"] = result["mfg"]
                mini_dict["category"] = result["category"]
                item_info.append(mini_dict)

    return item_info 
      
"""



results_dict = json.loads(page_soup.text)


item_info = []
results_list = results_dict["results"]
for result in results_list:
    mini_dict = {}
    mini_dict["name"] = result["name"]
    mini_dict["company"] = result["mfg"]
    mini_dict["category"] = result["category"]
    item_info.append(mini_dict)

print(item_info)
    

In [27]:
#print(find_greenseal("Tile Cleaner"))

# NSF International by Company

"""
def find_NSF(company):

    # default input to similarity function, only need to tokenize and find stopwords once
    X_list = word_tokenize(company) 

    company = company.replace(" ","%20")
    my_url = f"http://info.nsf.org/Certified/Common/Company.asp?CompanyName={company}&x=0&y=0"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"

    headers = {"user-agent":user_agent}
    resp = requests.get(my_url, headers=headers)

    if resp.status_code == 200:
        page_soup = soup(resp.content, "html.parser")
    else:
        return None
    
    if "No Matching Manufacturers Found" in page_soup.text:
        return None
    
    content_area = page_soup.findAll("div",{"id":"content-area"})[0]
    items = content_area.findAll("table")
    comps_and_cats = []
    for item in items:
        company_name = str(item.td.a.strong.font).split(">")[1].split("</")[0].strip()
        category = str(item.td).split("<br/>")[1].strip("</td>").strip().replace("\r","").replace("\n","").replace("  ","")
        
        if len(comps_and_cats)<=1:
            comps_and_cats.append([company_name,category])
        else:
            similarity_score = check_nlp(company_name,X_list,company)
            allowable = 0.3+0.1*(len(comps_and_cats))
            if similarity_score >= allowable:
                omps_and_cats.append([company_name,category])
        
    return comps_and_cats
"""

content_area = page_soup.findAll("div",{"id":"content-area"})[0]
items = content_area.findAll("table")

comps_and_cats = []
for item in items:
    company_name = str(item.td.a.strong.font).split(">")[1].split("</")[0].strip()
    category = str(item.td).split("<br/>")[1].strip("</td>").strip().replace("\r","").replace("\n","").replace("  ","")
    comps_and_cats.append([company_name,category])

## Fair Trade Scraping

In [28]:
def check_fair_trade(company,check_parts=True,fb_connect=fb_connect):
    
    # first check firebase
    result = fb_connect.get(f"/brand-Fair Trade/{company}",None)
    if result is not None:
        return int(result)

    company = company.replace(" ","+")
    fair_trade_url = f"https://www.fairtradecertified.org/search/fair-trade-products?product_type=All&name={company}"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
    company = company.replace("+"," ")
    
    headers = {"user-agent":user_agent}
    resp = requests.get(fair_trade_url, headers=headers)

    if resp.status_code==200:
        page_soup = soup(resp.content,"html.parser")

    companies = page_soup.findAll("h3")
    if len(companies)==0:
        if check_parts is False:
            z = 0
        else:
            z = max(check_fair_trade(x.strip(),check_parts=False) for x in company.split())
    else:
        z=1
    
    result = fb_connect.put("/brand-Fair Trade/",company,z)
    return z

    # default input to similarity function, only need to tokenize and find stopwords once
   # X_list = word_tokenize(company) 

   # for company in companies:
    #    company = company.text
     #   similarity_score = check_nlp(company,X_list,company)
      #  if similarity_score >= 0.35:
       #     return 1



In [29]:
#print(check_fair_trade("Test"))

In [30]:
now = datetime.datetime.now()
fair_trade = check_fair_trade(brand)
all_product_values["fair trade"] = fair_trade
later = datetime.datetime.now()
duration = later- now
time_dict["fair trade"] = duration

In [31]:
#print(brand,fair_trade)

# ECHO API 

In [32]:
def evaluate_facility(mini_dict):
    status = mini_dict["CURR_COMP_STATUS"]
    if "serious" in status:
        return 2
    elif "(s)" in status:
        return 1
    else:
        return 0
    
def violation_score(return_list):
    score=0
    for mini_dict in return_list:
        score+=evaluate_facility(mini_dict)
    return score

def ECHO_violations(brand):
    brand = brand.upper()
    api_url = f"https://enviro.epa.gov/enviro/efservice/t_compliance_echo/NAME/CONTAINING/{brand}/JSON"
    response = requests.get(api_url)
    if response.status_code == 200:
        return_list = json.loads(response.text)
        if len(return_list)==0:
            return 0
        else:
            vio_ratio = violation_score(return_list)/len(return_list)
            return vio_ratio*10
    else:
        return None



In [33]:

now = datetime.datetime.now() #

echo = ECHO_violations(brand)
echo+= ECHO_violations(owner_company)

later = datetime.datetime.now() #
duration = later- now
time_dict["ECHO"] = duration #

echo = endpoints(echo,lower=0,upper=5)

echo *= (0-1)

all_product_values["echo"] = echo



In [34]:
#print(echo)

## Non-GMO Project Check 
still need to rewrite so that will check brand, not product

In [35]:
def nonGMO(product,brand,first_try=True):
    
    recurse = False # initialize value
    api_url = f"https://ws2.nongmoproject.org/api/v1/get_brands_products_by_keyword?keyword={product}&page=1"
    response = requests.get(api_url)
    if response.status_code==200:
        results_dict = json.loads(response.text)
        results_list = results_dict["data"]
    else:
        recurse = True
    
    if len(results_list)==0 or recurse is True:
        
        if first_try:
            if brand == "" and len(product.split())<=2:
                for x in brand.split():
                    if nonGMO(x,"none",first_try=False)==1:
                        return 1
            elif brand!="":
                return nonGMO(brand,"")
        else:
            return 0

    # default input to similarity function, only need to tokenize and find stopwords once
    X_list = word_tokenize(product) 

    for match in results_list:
        name = match["name"]
        similarity_score = check_nlp(name,X_list,product)
        if similarity_score >= 0.35:
            return 1


In [36]:
#print(product)

In [37]:
now = datetime.datetime.now() #

nGMO = nonGMO(product,brand)
all_product_values["nonGMO"] = nGMO


later = datetime.datetime.now() #
duration = later - now
time_dict["nonGMO"] = duration #

In [38]:
#print(nGMO)

## CDP A-List

In [39]:
def CDP_A(company):

    with open("cdp_dict.json","r") as json_file:
        cdp_dict = json.loads(json_file.readline())
    
    possible = []
    company = company.lower()
    
    for key in cdp_dict.keys():
        if company in key.lower() or key.lower() in company:
            possible.append([key,cdp_dict[key]])
    if len(possible)==0:
        return 0
    elif len(possible)==1:
        return possible[0][1]
    else:
        # default input to similarity function, only need to tokenize and find stopwords once
        X_list = word_tokenize(company) 


        round2_dict = {}
        for option in possible:
            sim_score = check_nlp(option[0],X_list,company)
            round2_dict[sim_score] = option
        best_list = round2_dict[max(round2_dict.keys())]
        return best_list[1]

In [40]:
now = datetime.datetime.now()


cdpA = CDP_A(brand)
cdpA+= CDP_A(owner_company)/2
cdpA = endpoints(cdpA,lower=0,upper=3)
all_product_values["cdpA"] = cdpA

later = datetime.datetime.now()
duration = later- now
time_dict["cdpA"] = duration

In [41]:
# print(cdpA)

# Rainforest Alliance

In [42]:
def ra_check(brand,fb_connect=fb_connect):
    
    result = fb_connect.get(f"/brand-RA/{brand}",None)
    if result is not None:
        return result
    
    # request module and parsing
    ra_url = f"https://www.rainforest-alliance.org/find-certified?location=330&category=&keyword={brand}&op=submit"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"

    headers = {"user-agent":user_agent}
    resp = requests.get(ra_url, headers=headers)

    if resp.status_code!=200:
        return None
    else:
        page_soup = soup(resp.text,"html.parser")
        
    companies = page_soup.findAll("div",{"class":"cp-teaser-info-content"})
    
    if len(companies)==0:
        z = 0
    else:
        z = 1
    
    result2 = fb_connect.put("/brand-RA/",brand,z)
    return z
    
    #useful_dict = {}
    #for company in companies:
     #   return 1
  #  return 0
        
        #products = company.div.findAll("span")
        #cleaned_products = []
        #for product in products:
        #    if products.index(product)!=0:
         #       cleaned_products.append(product.text.strip().strip(","))
        #useful_dict[comp_name] = cleaned_products
        
        
 #   return useful_dict

In [43]:
now = datetime.datetime.now()

RA = ra_check(brand)
RA += ra_check(owner_company)/2
all_product_values["rainforest alliance"] = RA


later = datetime.datetime.now()
duration = later- now
time_dict["rainforest alliance"] = duration

In [44]:
print(RA)

0.0


def find_ingredients(product,brand,company):

    all_results=[]
    my_api_key = "9ZTbuCeX9dP5keqlGJZyWNdg1c1xCu3p9pLhJ1MS"
    query = product.replace(" ","%20") # format more friendly
    # make request with product name, brand owner, limit to 25 results
    response = requests.get(f"https://api.nal.usda.gov/fdc/v1/foods/search?api_key={my_api_key}&query={query}&brandOwner={brand}&pageSize=25")
    big_dict = response.json() # json format
    foods = big_dict["foods"] # list of all foods that match result
    
    # if less than 10 foods match search with that brand, check again replacing brand with company
    if len(foods)<10:
        response2 = requests.get(f"https://api.nal.usda.gov/fdc/v1/foods/search?api_key={my_api_key}&query={query}&brandOwner={company}&pageSize=25")
        big_dict_2 = response.json()
        foods2 = big_dict["foods"]
        foods.extend(foods2)
    
    ingredients = set([]) #initialize empty set
    for food in foods:
        if "ingredients" in food.keys(): # if food has ingredients, get the string
            string_mini_ingred = food["ingredients"]
            lst_mini_ingred = string_mini_ingred.split(',')
            for term in lst_mini_ingred:
                halves = term.split("(") # splitting by commas leaves elements with parantheses, so split further
                for half in halves:
                    if "FLAVOR" not in half: # get rid of "Natural and Artifical flavor", "maintains flavor", etc
                        ingredients.add(half.strip().strip(".").strip(")")) #clean up and add to ingredients set


    return ingredients
            
        #keys = food.keys()
        #if "ingredients" in keys:
            #ingred_str = food["ingredients"]
            #all_matches.append({"name":food["description"],"ingredients":ingred_str})
    #return all_matches

In [45]:
## creating final score

final_score = 5 # initialize

used = 0 # num of keys that were not none
keys = ["organic","fair trade","echo","nonGMO","cdpA","rainforest alliance"]
#keys.append("ingredient score")
for key in keys:
    temp_val = all_product_values[key]
    if temp_val != None:
        final_score+=temp_val
        used+=1
        
final_score = final_score / (used/len(keys)) # scale up score to take into account to avoid punishing for None
        
final_score = endpoints(final_score,lower=0,upper=10)

all_product_values["pre-score"] = final_score

all_product_values["Eco-Score"] = round(final_score)


In [46]:
print(all_product_values)

{'brand': 'Pepperidge Farm', 'product name': 'Goldfish', 'owner company': 'Campbell Soup Company', 'organic operation ID': 'None', 'organic': 0, 'fair trade': 1, 'echo': -2.0, 'nonGMO': None, 'cdpA': 0.0, 'rainforest alliance': 0.0, 'pre-score': 4.8, 'Eco-Score': 5}


In [47]:
time_dict_2 = {}
for key in time_dict.keys():
    time_dict_2[key] = str(time_dict[key])
print(time_dict_2)

{'imports': '0:00:01.141998', 'lookup upc': '0:00:00.451647', 'owner company': '0:00:00.905563', 'creating opID_dict': '0:00:00.060752', 'find opID': '0:00:00.216261', 'fair trade': '0:00:00.213311', 'ECHO': '0:00:03.150667', 'nonGMO': '0:00:00.508218', 'cdpA': '0:00:00.004653', 'rainforest alliance': '0:00:00.425389'}


with open(f"v1_{query}_times.json","w") as file:
    string = json.dumps(time_dict_2)
    file.write(string)